In [1]:
#!pip3 install pandas 
#!pip install --upgrade pip

In [2]:
#!pip install --upgrade --user pip

In [3]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [4]:
df.loc[:,['Gold', 'Gold.1', 'Gold.2']].head()

,Gold,Gold.1,Gold.2
Afghanistan,0,0,0
Algeria,5,0,5
Argentina,18,0,18
Armenia,1,0,1
Australasia,3,0,3


In [5]:
df[df['Gold.2'] == max(df['Gold.2'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


In [6]:
df[df['Gold.2'] == max(df['Gold.2'])].index[0]

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [40]:
difGG1 = abs(df['Gold'] - df['Gold.1'])

In [43]:
difGG1.head()

Afghanistan     0
Algeria         5
Argentina      18
Armenia         1
Australasia     3
dtype: int64

In [44]:
max(difGG1)

880

In [47]:
df[df['Gold'] - df['Gold.1'] == max(difGG1)]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


In [53]:
df[df['Gold'] - df['Gold.1'] == max(difGG1)].index[0]

'United States'

In [63]:
difGG1[difGG1 == max(difGG1)].index[0]

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [69]:
difGG1TR = difGG1 / df['Gold.2']

In [89]:
max(difGG1TR.where(difGG1TR < 1).dropna())

0.9615384615384616

In [92]:
difGG1TR[difGG1TR == max(difGG1TR.where(difGG1TR < 1).dropna())].index[0]

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

### Ejercicios DEA

In [9]:
## ¿Cuántos países tienen Gold?
df[df['Gold']>0]['Gold'].count()

99

In [10]:
## ¿Cuántos países tienen Gold.1?
df[df['Gold.1']>0]['Gold.1'].count()

37

In [11]:
df[df['Gold.1']>0]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Australia,25,139,152,177,468,18,5,3,4,12,43,144,155,181,480,AUS
Austria,26,18,33,35,86,22,59,78,81,218,48,77,111,116,304,AUT
Belarus,5,12,24,39,75,6,6,4,5,15,11,18,28,44,90,BLR
Belgium,25,37,52,53,142,20,1,1,3,5,45,38,53,56,147,BEL
Bulgaria,19,51,85,78,214,19,1,2,3,6,38,52,87,81,220,BUL
Canada,25,59,99,121,279,22,62,56,52,170,47,121,155,173,449,CAN
China,9,201,146,126,473,10,12,22,19,53,19,213,168,145,526,CHN
Croatia,6,6,7,10,23,7,4,6,1,11,13,10,13,11,34,CRO
Czech Republic,5,14,15,15,44,6,7,9,8,24,11,21,24,23,68,CZE
Czechoslovakia,16,49,49,45,143,16,2,8,15,25,32,51,57,60,168,TCH


In [12]:
## ¿Cuántos países tienen Gold y Gold.1?
## Países que han ganado medalla de oro en Summer y Winter
len(df[(df['Gold']>0) & (df['Gold.1']>0)])

36

In [13]:
df[(df['Gold']>0) & (df['Gold.1']>0)].head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Australia,25,139,152,177,468,18,5,3,4,12,43,144,155,181,480,AUS
Austria,26,18,33,35,86,22,59,78,81,218,48,77,111,116,304,AUT
Belarus,5,12,24,39,75,6,6,4,5,15,11,18,28,44,90,BLR
Belgium,25,37,52,53,142,20,1,1,3,5,45,38,53,56,147,BEL
Bulgaria,19,51,85,78,214,19,1,2,3,6,38,52,87,81,220,BUL


In [14]:
##Países que han ganado medalla de oro en Summer y no en Winter
len(df[(df['Gold']>0)&(df['Gold.1']==0)])

63

In [15]:
df[(df['Gold']>0)&(df['Gold.1']==0)].head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ
Azerbaijan,5,6,5,15,26,5,0,0,0,0,10,6,5,15,26,AZE


In [16]:
## Países que han ganado medalla de oro en Winter y no en Summer
df[(df['Gold']==0)&(df['Gold.1']>0)]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Liechtenstein,16,0,0,0,0,18,2,2,5,9,34,2,2,5,9,LIE


In [17]:
len(df[(df['Gold']==0)&(df['Gold.1']>0)])

1

In [18]:
##Máximo ganador de oro Winter
df[df['Gold.1'] == max(df['Gold.1'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Norway,24,56,49,43,148,22,118,111,100,329,46,174,160,143,477,NOR


In [19]:
##¿Quién ha ido más veces a Winter?
df[df['# Winter'] == max(df['# Winter'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Austria,26,18,33,35,86,22,59,78,81,218,48,77,111,116,304,AUT
Canada,25,59,99,121,279,22,62,56,52,170,47,121,155,173,449,CAN
Finland,24,101,84,117,302,22,42,62,57,161,46,143,146,174,463,FIN
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
Hungary,25,167,144,165,476,22,0,2,4,6,47,167,146,169,482,HUN
Italy,26,198,166,185,549,22,37,34,43,114,48,235,200,228,663,ITA
Norway,24,56,49,43,148,22,118,111,100,329,46,174,160,143,477,NOR
Poland,20,64,82,125,271,22,6,7,7,20,42,70,89,132,291,POL
Sweden,26,143,164,176,483,22,50,40,54,144,48,193,204,230,627,SWE


In [20]:
##¿Quién ha ido más veces a Summer?
df[df['# Summer'] == max(df['# Summer'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
Greece,27,30,42,39,111,18,0,0,0,0,45,30,42,39,111,GRE
Switzerland,27,47,73,65,185,22,50,40,48,138,49,97,113,113,323,SUI


In [21]:
##Países que más han ido a Summer y Winter
df[(df['# Summer'] == max(df['# Summer']))&(df['# Winter'] == max(df['# Winter']))]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
Switzerland,27,47,73,65,185,22,50,40,48,138,49,97,113,113,323,SUI


In [24]:
##Máximo ganador de medallas
df[df['Combined total'] == max(df['Combined total'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


In [25]:
##Máximo ganador de medallas en Summer
df[df['Total'] == max(df['Total'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


In [26]:
##Máximo ganador de medallas en Winter
df[df['Total.1'] == max(df['Total.1'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Norway,24,56,49,43,148,22,118,111,100,329,46,174,160,143,477,NOR


In [30]:
##Países con 0 medallas totales
df[df['Combined total'] == 0]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID


In [31]:
##País con menos medallas totales
df[df['Combined total'] == min(df['Combined total'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Bahrain,8,0,0,1,1,0,0,0,0,0,8,0,0,1,1,BRN
Barbados,11,0,0,1,1,0,0,0,0,0,11,0,0,1,1,BAR
Bermuda,17,0,0,1,1,7,0,0,0,0,24,0,0,1,1,BER
Botswana,9,0,1,0,1,0,0,0,0,0,9,0,1,0,1,BOT
Burundi,5,1,0,0,1,0,0,0,0,0,5,1,0,0,1,BDI
Ivory Coast,12,0,1,0,1,0,0,0,0,0,12,0,1,0,1,CIV
Cyprus,9,0,1,0,1,10,0,0,0,0,19,0,1,0,1,CYP
Djibouti,7,0,0,1,1,0,0,0,0,0,7,0,0,1,1,DJI
Eritrea,4,0,0,1,1,0,0,0,0,0,4,0,0,1,1,ERI
Gabon,9,0,1,0,1,0,0,0,0,0,9,0,1,0,1,GAB


In [52]:
##Promedio de medallas totales ganadas
df['Combined total'].mean()

120.4041095890411

In [103]:
df.sort_values(by=['Combined total'], ascending=False).head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA
Soviet Union,9,395,319,296,1010,9,78,57,59,194,18,473,376,355,1204,URS
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
Germany,15,174,182,217,573,11,78,78,53,209,26,252,260,270,782,GER
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA


In [109]:
df.loc['Russia'] + df.loc['Soviet Union']

# Summer              14
Gold                 527
Silver               440
Bronze               438
Total               1405
# Winter              15
Gold.1               127
Silver.1              97
Bronze.1              94
Total.1              318
# Games               29
Gold.2               654
Silver.2             537
Bronze.2             532
Combined total      1723
ID                RUSURS
dtype: object

In [111]:
df.loc['Russia']

# Summer            5
Gold              132
Silver            121
Bronze            142
Total             395
# Winter            6
Gold.1             49
Silver.1           40
Bronze.1           35
Total.1           124
# Games            11
Gold.2            181
Silver.2          161
Bronze.2          177
Combined total    519
ID                RUS
Name: Russia, dtype: object

In [112]:
df.loc['Soviet Union']

# Summer             9
Gold               395
Silver             319
Bronze             296
Total             1010
# Winter             9
Gold.1              78
Silver.1            57
Bronze.1            59
Total.1            194
# Games             18
Gold.2             473
Silver.2           376
Bronze.2           355
Combined total    1204
ID                 URS
Name: Soviet Union, dtype: object